# Resumen de métricas y conclusiones

## Resumen EDA

Datos desbalanceados entre categorías (algunas clases con mucho menos soporte).

Textos largos: los CVs son extensos; p90 de tokens alto → los modelos sensibles a la longitud agradecen max_len amplios (p. ej. 512).

Vocabulario técnico y términos de dominio frecuentes (banca, IT, salud, etc.).

N-gramas muestran palabras y frases distintivas por clase; ayudan a enfoques lineales (TF-IDF) y a interpretar errores.

## Detalles de los modelos.

### 1 DistilBERT (fine-tuning)

Arquitectura: Transformer preentrenado para clasificación (capa softmax sobre [CLS]).

Entrenamiento: max_len = 512, batch ≈ 16, lr = 2e-5, epochs = 10 con early stopping (patience=2), class weights activados.

Resultado (macro): Acc 0.8755, Prec 0.8477, Rec 0.8560, F1 0.8440 (mejor global y más balanceado).


### 2 TF-IDF + XGBoost (GPU)

Arquitectura: bolsa de palabras ponderada + árbol de gradient boosting multinomial.

Entrenamiento: GPU (tree_method=gpu_hist), early stopping; (ver notebook para hiperparámetros exactos).

Resultado (macro): Acc 0.8032, F1 0.7847.


### 3 CNN-1D

Arquitectura: Embedding → Conv1D + Pooling → Dense (clasificación).

Resultado (macro): Acc 0.8153, F1 0.7658.


### 4 Word2Vec + BiLSTM

Arquitectura: Embeddings Word2Vec + BiLSTM.

Mejor config (grid): lstm_units=128, num_layers=1, dropout=0.3, lr=0.005, batch=64, seq_len=100, epochs=50, emb_dim=100.

Resultado (macro): Acc 0.7149, F1 0.6554.

### 5 FastText

Arquitectura: bag-of-ngrams + promedios; muy rápido.

Resultado (macro): Acc 0.5221, F1 0.4738 (baseline)

## Métricas

In [5]:
import pandas as pd
from IPython.display import display

# Tabla de métricas (macro) — con AUC macro OvR incluido
rows = [
    {
        "Modelo": "DistilBERT (fine-tuned)",
        "Accuracy": 0.8755,
        "Precision (macro)": 0.8477,
        "Recall (macro)": 0.8560,
        "F1 (macro)": 0.8440,
        "ROC-AUC (macro OvR)": 0.9828,
    },
    {
        "Modelo": "TF-IDF + XGBoost (GPU)",
        "Accuracy": 0.8032128514056225,
        "Precision (macro)": 0.8142526182967359,
        "Recall (macro)": 0.782807239057239,
        "F1 (macro)": 0.7846779362500778,
        "ROC-AUC (macro OvR)": 0.9866,
    },
    {
        "Modelo": "CNN-1D",
        "Accuracy": 0.8152610441767069,
        "Precision (macro)": 0.7839116901616903,
        "Recall (macro)": 0.7761994949494949,
        "F1 (macro)": 0.7657776426683749,
        "ROC-AUC (macro OvR)": 0.9820,
    },
    {
        "Modelo": "FastText (supervised)",
        "Accuracy": 0.5220883534136547,
        "Precision (macro)": 0.5132000063564709,
        "Recall (macro)": 0.4989267676767677,
        "F1 (macro)": 0.4737797239462891,
        "ROC-AUC (macro OvR)": 0.8309,   # tu valor calculado
    },
    {
        "Modelo": "Word2Vec + BiLSTM",
        "Accuracy": 0.714859437751004,
        "Precision (macro)": 0.6381595256595257,
        "Recall (macro)": 0.6868686868686869,
        "F1 (macro)": 0.655387690618402,
        "ROC-AUC (macro OvR)": 0.9398,
    },
]

df = pd.DataFrame(rows)

# Ordena por F1 (macro) desc
df = df.sort_values("F1 (macro)", ascending=False).reset_index(drop=True)

# Redondea a 3 decimales y muestra
metric_cols = ["Accuracy", "Precision (macro)", "Recall (macro)", "F1 (macro)", "ROC-AUC (macro OvR)"]
df[metric_cols] = df[metric_cols].astype(float).round(3)
display(df)



,Modelo,Accuracy,Precision (macro),Recall (macro),F1 (macro),ROC-AUC (macro OvR)
0,DistilBERT (fine-tuned),0.876,0.848,0.856,0.844,0.983
1,TF-IDF + XGBoost (GPU),0.803,0.814,0.783,0.785,0.987
2,CNN-1D,0.815,0.784,0.776,0.766,0.982
3,Word2Vec + BiLSTM,0.715,0.638,0.687,0.655,0.940
4,FastText (supervised),0.522,0.513,0.499,0.474,0.831


DistilBERT es posible que gane ya que aprovecha contexto largo (512 tokens), entiende semántica y relaciones no lineales; los class weights y early stopping ayudaron a la estabilidad.

TF-IDF + XGBoost no se queda atrás ya que los CVs tienen señales léxicas fuertes por dominio; los árboles capturan bien interacciones de n-gramas a bajo coste.

Limitaciones observadas: desbalance por clase, posible pérdida de contexto en modelos con seq_len pequeño, ruido en los CVs.

## Conclusiones

Para este dataset, DistilBERT fine-tuned es la mejor opción cuando priorizamos F1 macro y desempeño equilibrado por clase. TF-IDF + XGBoost (GPU) es una alternativa eficiente con rendimiento cercano y menor coste. Con limpieza de datos, balanceo y (idealmente) preentrenamiento adaptado al dominio, es razonable esperar mejoras adicionales en recall de clases minoritarias y F1 macro.

## Recomendaciones para mejora


Entre los problemas que se encuentran en el dataset es el hecho de que está desbalanceado, a lo que una  de las posibles soluciones es el uso de métodos como el oversampling teniendo en cuenta el riesgo de overfitting. También se le podría dar mas peso a las clases minoritarias al momento de entrear el modelo. Estas opciones podrían ayudar a que el modelo pueda entender un poco mas la naturaleza del dataset y por consiguiente, poder mejorar las métricas.

Asímismo, también se puede optar por usar otros modelos que tal vez puedan comprender mejor el significado de las frases dentro de los contextos dados y con ello, una posible solución para abarcar los problemas es la inclusión de un modelo híbrido que obtenga "diferentes puntos de vista" o diferentes maneras de analizar el problema para darle mas versatilidad.